In [24]:
import torch.nn as nn
import torch
import torch.nn.functional as F
import torch.distributions as distributions
# m = nn.Softmax()
# input = torch.tensor([-1, 0., 1.])
# output = m(input)
# print(output)
# input = torch.tensor([0, 1, 0, 1])
# F.one_hot(input, num_classes=5).shape
# probs = torch.tensor([[0.2,0.3,0.5],[0.5,0.5,0.5]])
# logits = torch.log(probs)
# # distr = distributions.Categorical(probs=logits)
# # out = distr.sample()
# out=torch.multinomial(torch.exp(logits), 5,replacement=True)
# print(out)
# input = torch.randint(0, 3, (5,3), dtype=torch.int64)
# input=torch.tensor([[0, 2, 0],
#         [2, 0, 0],
#         [2, 2, 2],
#         [2, 1, 1],
#         [1, 1, 1]])
# batch_size, count = input.shape
# temp = torch.arange(batch_size) * count
# temp = temp[:,None].expand(input.shape)
# print(input, input+temp)
# torch.bincount((input+temp).reshape(-1), minlength=count*batch_size).reshape([batch_size,-1])
# y = input = torch.randint(0, 3, (5,3), dtype=torch.int64)
# print(y)
# torch.tile(y, (3,)).reshape([-1]+list (y.shape[1:]))
# m = nn.Identity()
# input = torch.randn(2, 4)
# output = m(input)
# print(input, output)
# torch.zeros((2,3,4)).shape
input = torch.randint(0, 3, (5,3), dtype=torch.int64)
torch.repeat_interleave(torch.arange(15), input.reshape(-1)).shape
# torch.normal(0, 1,size=input.shape).shape
# no_grad = torch.tensor(3.0)
# x = no_grad.clone().detach().requires_grad_(True)
# # with tf.GradientTape() as g:
# #   g.watch(x)
# y = torch.exp(x) * x
# y.backward()
# print(x.grad.data,x, no_grad.requires_grad)
# y_true = [[0, 1], [0, 0]]
# y_pred = [[0.6, 0.4], [0.4, 0.6]]
# kl_numpy = np.array(y_true)*np.log(np.array(y_true)/np.array(y_pred) + 1e-10)
# print(kl_numpy)
# print(kl_numpy.sum(axis=-1))
# kl = nn.KLDivLoss(reduction='none')
# kl( (torch.tensor(y_pred)).log(), (torch.tensor(y_true)+1e-10)).sum(-1)

torch.Size([21])

In [26]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '3'
import tensorflow as tf
import numpy as np
# indices = [0, 1, 0, 1]
# depth = 5
# a=tf.one_hot(indices, depth)
# print(a.shape)
# samples = tf.random.categorical(tf.math.log([[0.2,0.3,0.5],[0.5,0.5,0.5]]), 5)
# print(samples)
# tf.identity(a)
# tf.random.normal(a.shape).shape
# x = tf.constant(3.0)
# with tf.GradientTape() as g:
#   g.watch(x)
#   z = tf.exp(x)
#   y = z*z
# dy_dx = g.gradient(y, x)
# print(dy_dx,x)
# x *= x
# with tf.GradientTape() as g:
#   g.watch(x)
#   y = x * x
# dy_dx = g.gradient(y, x)
# print(dy_dx)

# y_true = [[0, 1], [0, 0]]
# y_pred = [[0.6, 0.4], [0.4, 0.6]]
# kl_numpy = np.array(y_true)*np.log(np.array(y_true)/np.array(y_pred) + 1e-10)

# print(np.array(y_true)*np.log(np.array(y_true)/np.array(y_pred)+ 1e-10))
# print(kl_numpy.sum(axis=-1))
# # Using 'auto'/'sum_over_batch_size' reduction type.
# kl = tf.keras.losses.KLDivergence()
# kl(y_true, y_pred).numpy()
# kl = tf.keras.losses.KLDivergence(
#     reduction=tf.keras.losses.Reduction.NONE)
# kl(y_true, y_pred).numpy()
batch_size = 2
num_action_samples = 2048
actions_selected = tf.random.categorical([4096])
repeat_indices = tf.repeat(
        tf.range(batch_size * num_action_samples), actions_selected)
print(repeat_indices.shape)

AttributeError: module 'tensorflow._api.v2.random' has no attribute 'randn'

In [22]:
from environments.particle.particle import ParticleEnv
import gym
import torch
env = ParticleEnv()
batch_size=2
num_action_samples=2048
obs = env.reset()
# obs = torch.tensor(time_step)[None, Ellipsis]
# obs = torch.concat([obs] * (batch_size * num_action_samples),
#                             axis=0)
# "Batch" the observations by replicating
for key in obs.keys():
    batch_obs = torch.tensor(obs[key])[None, Ellipsis]
    obs[key] = torch.concat([batch_obs] * (batch_size * num_action_samples),axis=0)
print(type(env.observation_space.spaces))
num_dict=1
if isinstance(env.observation_space.spaces, dict):
    num_dict = len(env.observation_space.spaces.keys())
print(num_dict)
# env = suite_gym.wrap_env(env)
# obs_spec, act_spec, _ = spec_utils.get_tensor_specs(env)
# for key in list(env.observation_space.spaces):
# env = gym.make("InvertedPendulum-v2")
# obs_dim = env.observation_space.shape[0]
# act_dim = env.action_space.shape[0]
# print("obs space", env.observation_space, "action space",env.action_space)
# print("obs space", obs_dim, "action space",act_dim)

<class 'collections.OrderedDict'>
4


In [13]:
import torch
import numpy as np
time_step = np.random.rand(4)
batch_size = 2
num_action_samples = 2048
print("timestep", time_step)
obs = torch.tensor(time_step)[None, Ellipsis]
obs = torch.concat([obs] * (batch_size * num_action_samples),
                        axis=0)
print(obs.shape)

timestep [0.54878577 0.6373976  0.36660476 0.84392894]
torch.Size([4096, 4])


In [1]:
from stable_baselines3.common.evaluation import evaluate_policy
import gym

from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecNormalize

env = make_vec_env("HalfCheetah-v3", n_envs=20, seed=0)
# env = VecNormalize(env, training=False, norm_reward=False)
# env = gym.make("HalfCheetah-v3",)
model = PPO.load('baseline_ppo_{i}.pth')
mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=10)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

KeyboardInterrupt: 